In [2]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")

KT_vars = ['x','y','z1','z2','z3','z4','z5','v1','v2','v3','v4','v5']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m','l1','l2','l3','l4','l5','w1','w2','w3','w4','w5']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)

poly_ring = PolynomialRing(ZZ, coho_vars)


x,y,z1,z2,z3,z4,z5,v1,v2,v3,v4,v5=KT_ring.gens()
t,m,l1,l2,l3,l4,l5,w1,w2,w3,w4,w5= poly_ring.gens()
b=var('b')
'''   w=SR.var('w',5)   '''
v=[v1,v2,v3,v4,v5]
w=[w1,w2,w3,w4,w5]
L=var('l')
z=[z1,z2,z3,z4,z5]


def character(p,x=x,y=y):
    return sum(y^(-r) * x^(-c) for r, c in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))

def dual(f):
    R = f.parent()
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def tangent(p,x=x,y=y):
    total=0
    for r,c in p.cells():
        arm=p[r]-c-1
        leg=p.conjugate()[c]-r-1
        total+=y^(-leg)*x^(arm+1)+y^(leg+1)*x^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)


def structure_sheaf(p,x=x,y=y):
    return character_vb(p, (0,0),x,y)

def top_chern(f):
    numer, denom = 1, [1]
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer *= wt^coeff
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return [numer,denom]

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * poly_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def measure_unsymmetrized(f, inv=True):
    """
    computes 1/(1-x^-1) for K-thy class x. If inv=false, returns 1/(1-x)
    """
    R = f.parent()
    
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)

    numer, denom = R.one(), R.one()
    for coeff, monomial in L:
        if inv:
            term = 1 - monomial^-1
        else:
            term = 1 - monomial
        if monomial==1:
            term = 0
        if coeff < 0:
            numer *= term ** (-coeff)
        elif coeff > 0:
            denom *= term ** coeff
    return numer / denom



def degn(f,n,b=b):
    g=f.taylor(b,0,n+2)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

    

In [7]:
def partitions(Sum, K):
    '''computes fixed components of Quot by CC^* action.'''
    total=[]
    def ptn(Sum, K, lst, total):
        if K == 0:
            if Sum == 0:
                total+=[lst]
            return
        for i in range(0, min(Sum + 1, Sum + 1)):
            ptn(Sum - i, K - 1, lst + [i],total)
    ptn(Sum,K,[],total)
    return total


def fixed_locus(n, N):
    '''gives list of fixed points of Hilb^n1 x Hilb^n2 x ... x Hilb^nN, input (n1,n2,...,nN)'''
    total=[]
    ptns=partitions(n,N)
    for j in range(len(ptns)):
        ptn=ptns[j]
        lst=[]
        for i in range(N):
            lst+=[Partitions(ptn[i]).list()]
        total+=[p for p in itertools.product(*lst)]
    return total

def Nvir(Z):
    '''virtual normal bundle of quot scheme at (Z_1,...,Z_N)'''
    total=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            print(pretty((1-character(Z[i],x=x^-1,y=y^-1)*(1-x)*(1-y))*character(Z[j],x=x,y=y)*(z[j]/z[i])))
            total+=(1-character(Z[i],x=x^-1,y=y^-1)*(1-x)*(1-y))*character(Z[j],x=x,y=y)*(z[j]/z[i])
    print("!!"+str(pretty(total)))
    return total


def total_chern_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((1+r) for r in roots)
    return total

def top_chern_quot(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((r) for r in roots)
    return total

def Chern_Num_quot(n,N,rank):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_chern_quot(Z,rank)
        total+=add
    return total

def DT_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))
        total+=add
    return total

def total_segre_quot(Z,rank):
    total_denom=[1]
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total_denom += [1+r for r in roots]
    return [1,total_denom]

def Segre_Num_quot(n,N,rank):
    total=[0,[1]]
    for Z in fixed_locus(n,N):
        add=mult_poly(top_chern(-Nvir(Z)),total_segre_quot(Z,rank))
        total= add_poly(total,add)
    return total

def Segre_Num_quot_sym(n,N,rank):
    total=[0,[1]]
    for Z in fixed_locus(n,rank):
        add=mult_poly(top_chern(-Nvir(Z)),total_segre_quot(Z,N))
        total= add_poly(total,add)
    return total


def determinant_quot(Z,n,rank):
    det=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        det *= determinant(Q)
    return (det^rank)*((prod(v[i] for i in range(rank)))^n)

def Verlinde_Num_quot(n,N,rank):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            determinant_quot(Z,n,rank)
        total+=add
    return total

def Verlinde_Num_quot_inv(n,N,rank):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            (1/determinant_quot(Z,n,rank))
    return total


def deg0(f):
    if f==0: 
        return 0
    return degn(f(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),0)

def deg02(f):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v),0))

def set_exp(f):
    for i in range(len(KT_vars)):
        f=SR(f).subs(KT_ring.gens()[i]==exp(coho_ring.gens()[i]))
    return f
def simplify_weight(f):
    return f(l1=0,l2=l1,l3=2*l1,l4=3*l1,l5=4*l5)

def symmetrize(f):
    if f in ZZ:
        return f
    return f(l1=w1,l2=w2,l3=w3,l4=w4,l5=w5,w1=l1,w2=l2,w3=l3,w4=l4,w5=l5)


def symmetrizeK(f):
    if f in ZZ:
        return f
    return f(z1=v1,z2=v2,z3=v3,z4=v4,z5=v5,v1=z1,v2=z2,v3=z3,v4=z4,v5=z5)

def all_check(start,finish):
    with open("symmetry2d-segre-checked.txt") as f:
        lines = f.read().splitlines()
    if len(lines)==0:
        n,N,r=[0,0,0]
    else:
        last_checked=lines[len(lines)-1]
        n,N,r=[int(a) for a in last_checked.split(",")]
    for i in range(start,finish+1):
        for j in range(1,3):
            for k in range(1,3):
                n,N,r=[i,j,k]
                if k<j:
                    continue
                if j<=k:
                    print("checking segre sym: n="+str(n)+", N="+str(N)+", r="+str(r))
                    if str(n)+','+str(N)+','+str(r) in lines:
                        print("n="+str(n)+", N="+str(N)+", r="+str(r)+" already checked")
                    else:
                        S1=Segre_Num_quot(n,N,r)
                        S2=Segre_Num_quot_sym(n,N,r)
                        diff=S1[0]*prod(S2[1])*(-1)^(n*N)-S2[0]*prod(S1[1])*(-1)^(n*r)
                        if diff==0:
                            writeline("symmetry2d-segre-checked.txt",str(n)+','+str(N)+','+str(r))
                            print("checked")
                        else:
                            print("failed")
        print("all checked for: n="+str(i))
               
                        
                    
def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
                
                
def clear_all_files():
    open('SV2d-checked.txt', 'w').close()
    open('symmetry2d-segre-checked.txt', 'w').close()
    open('symmetry2d-Verlinde-checked.txt', 'w').close()
    open('verlinde2d-out.txt', 'w').close()
    open('segre2d-out.txt', 'w').close()
    open('SV2d-out.txt', 'w').close()
            
def add_poly(f,g):
    numer=f[0]*prod(g[1])+g[0]*prod(f[1])
    denom=f[1]+g[1]
    return [numer,denom]

def mult_poly(f,g):
    numer=f[0]*g[0]
    denom=f[1]+g[1]
    return[numer, denom]
                                 
def symmetrize_poly(f):
    denom=[]
    for d in f[1]:
        if d in ZZ:
            denom+=[d]
        else:
            denom+=[d(l1=w1,l2=w2,l3=w3,l4=w4,l5=w5,w1=l1,w2=l2,w3=l3,w4=l4,w5=l5)]
    if f in ZZ:
        return f
    return [f[0](l1=w1,l2=w2,l3=w3,l4=w4,l5=w5,w1=l1,w2=l2,w3=l3,w4=l4,w5=l5),denom]
    

In [8]:
Segre_Num_quot(2,2,1)

0
(x + 1)*z2/(x*z1)
0
-(x^2*y - x^2 - y)*(x + 1)/x
!!-(x^2*y*z1 - x^2*z1 - y*z1 - z2)*(x + 1)/(x*z1)
0
(y + 1)*z2/(y*z1)
0
-(x*y^2 - y^2 - x)*(y + 1)/y
!!-(x*y^2*z1 - y^2*z1 - x*z1 - z2)*(y + 1)/(y*z1)
-x*y + x + y
-(x*y - x - y)*z2/z1
-(x*y - x - y)*z1/z2
-x*y + x + y
!!-(x*y - x - y)*(z1 + z2)^2/(z1*z2)
-(x^2*y - x^2 - y)*(x + 1)/x
0
(x + 1)*z1/(x*z2)
0
!!-(x^2*y*z2 - x^2*z2 - y*z2 - z1)*(x + 1)/(x*z2)
-(x*y^2 - y^2 - x)*(y + 1)/y
0
(y + 1)*z1/(y*z2)
0
!!-(x*y^2*z2 - y^2*z2 - x*z2 - z1)*(y + 1)/(y*z2)


[8*t^18*m^10*l1^6*l2^2 - 32*t^16*m^12*l1^6*l2^2 + 48*t^14*m^14*l1^6*l2^2 - 32*t^12*m^16*l1^6*l2^2 + 8*t^10*m^18*l1^6*l2^2 - 8*t^17*m^10*l1^7*l2^2 + 48*t^15*m^12*l1^7*l2^2 - 40*t^14*m^13*l1^7*l2^2 - 40*t^13*m^14*l1^7*l2^2 + 48*t^12*m^15*l1^7*l2^2 - 8*t^10*m^17*l1^7*l2^2 - 16*t^18*m^8*l1^8*l2^2 + 48*t^16*m^10*l1^8*l2^2 - 32*t^14*m^12*l1^8*l2^2 - 32*t^12*m^14*l1^8*l2^2 + 48*t^10*m^16*l1^8*l2^2 - 16*t^8*m^18*l1^8*l2^2 + 16*t^17*m^8*l1^9*l2^2 - 80*t^15*m^10*l1^9*l2^2 + 80*t^14*m^11*l1^9*l2^2 - 16*t^13*m^12*l1^9*l2^2 - 16*t^12*m^13*l1^9*l2^2 + 80*t^11*m^14*l1^9*l2^2 - 80*t^10*m^15*l1^9*l2^2 + 16*t^8*m^17*l1^9*l2^2 + 8*t^18*m^6*l1^10*l2^2 - 72*t^14*m^10*l1^10*l2^2 + 128*t^12*m^12*l1^10*l2^2 - 72*t^10*m^14*l1^10*l2^2 + 8*t^6*m^18*l1^10*l2^2 - 8*t^17*m^6*l1^11*l2^2 + 16*t^15*m^8*l1^11*l2^2 - 40*t^14*m^9*l1^11*l2^2 + 144*t^13*m^10*l1^11*l2^2 - 112*t^12*m^11*l1^11*l2^2 - 112*t^11*m^12*l1^11*l2^2 + 144*t^10*m^13*l1^11*l2^2 - 40*t^9*m^14*l1^11*l2^2 + 16*t^8*m^15*l1^11*l2^2 - 8*t^6*m^17*l1^11*l2^2 - 16*t^16*m^6*l1^12*l2^2 + 48*t^14*m^8*l1^12*l2^2 - 32*t^12*m^10*l1^12*l2^2 - 32*t^10*m^12*l1^12*l2^2 + 48*t^8*m^14*l1^12*l2^2 - 16*t^6*m^16*l1^12*l2^2 + 16*t^15*m^6*l1^13*l2^2 - 80*t^13*m^8*l1^13*l2^2 + 80*t^12*m^9*l1^13*l2^2 - 16*t^11*m^10*l1^13*l2^2 - 16*t^10*m^11*l1^13*l2^2 + 80*t^9*m^12*l1^13*l2^2 - 80*t^8*m^13*l1^13*l2^2 + 16*t^6*m^15*l1^13*l2^2 + 8*t^14*m^6*l1^14*l2^2 - 32*t^12*m^8*l1^14*l2^2 + 48*t^10*m^10*l1^14*l2^2 - 32*t^8*m^12*l1^14*l2^2 + 8*t^6*m^14*l1^14*l2^2 - 8*t^13*m^6*l1^15*l2^2 + 48*t^11*m^8*l1^15*l2^2 - 40*t^10*m^9*l1^15*l2^2 - 40*t^9*m^10*l1^15*l2^2 + 48*t^8*m^11*l1^15*l2^2 - 8*t^6*m^13*l1^15*l2^2 - 32*t^18*m^10*l1^5*l2^3 + 128*t^16*m^12*l1^5*l2^3 - 192*t^14*m^14*l1^5*l2^3 + 128*t^12*m^16*l1^5*l2^3 - 32*t^10*m^18*l1^5*l2^3 + 24*t^17*m^10*l1^6*l2^3 - 144*t^15*m^12*l1^6*l2^3 + 120*t^14*m^13*l1^6*l2^3 + 120*t^13*m^14*l1^6*l2^3 - 144*t^12*m^15*l1^6*l2^3 + 24*t^10*m^17*l1^6*l2^3 + 96*t^18*m^8*l1^7*l2^3 - 280*t^16*m^10*l1^7*l2^3 - 16*t^15*m^11*l1^7*l2^3 + 184*t^14*m^12*l1^7*l2^3 + 32*t^13*m^13*l1^7*l2^3 + 184*t^12*m^14*l1^7*l2^3 - 16*t^11*m^15*l1^7*l2^3 - 280*t^10*m^16*l1^7*l2^3 + 96*t^8*m^18*l1^7*l2^3 - 80*t^17*m^8*l1^8*l2^3 + 400*t^15*m^10*l1^8*l2^3 - 400*t^14*m^11*l1^8*l2^3 + 80*t^13*m^12*l1^8*l2^3 + 80*t^12*m^13*l1^8*l2^3 - 400*t^11*m^14*l1^8*l2^3 + 400*t^10*m^15*l1^8*l2^3 - 80*t^8*m^17*l1^8*l2^3 - 64*t^18*m^6*l1^9*l2^3 - 16*t^16*m^8*l1^9*l2^3 + 32*t^15*m^9*l1^9*l2^3 + 576*t^14*m^10*l1^9*l2^3 - 32*t^13*m^11*l1^9*l2^3 - 992*t^12*m^12*l1^9*l2^3 - 32*t^11*m^13*l1^9*l2^3 + 576*t^10*m^14*l1^9*l2^3 + 32*t^9*m^15*l1^9*l2^3 - 16*t^8*m^16*l1^9*l2^3 - 64*t^6*m^18*l1^9*l2^3 + 56*t^17*m^6*l1^10*l2^3 - 112*t^15*m^8*l1^10*l2^3 + 280*t^14*m^9*l1^10*l2^3 - 1008*t^13*m^10*l1^10*l2^3 + 784*t^12*m^11*l1^10*l2^3 + 784*t^11*m^12*l1^10*l2^3 - 1008*t^10*m^13*l1^10*l2^3 + 280*t^9*m^14*l1^10*l2^3 - 112*t^8*m^15*l1^10*l2^3 + 56*t^6*m^17*l1^10*l2^3 + 168*t^16*m^6*l1^11*l2^3 - 16*t^15*m^7*l1^11*l2^3 - 456*t^14*m^8*l1^11*l2^3 - 32*t^13*m^9*l1^11*l2^3 + 288*t^12*m^10*l1^11*l2^3 + 96*t^11*m^11*l1^11*l2^3 + 288*t^10*m^12*l1^11*l2^3 - 32*t^9*m^13*l1^11*l2^3 - 456*t^8*m^14*l1^11*l2^3 - 16*t^7*m^15*l1^11*l2^3 + 168*t^6*m^16*l1^11*l2^3 - 144*t^15*m^6*l1^12*l2^3 + 720*t^13*m^8*l1^12*l2^3 - 720*t^12*m^9*l1^12*l2^3 + 144*t^11*m^10*l1^12*l2^3 + 144*t^10*m^11*l1^12*l2^3 - 720*t^9*m^12*l1^12*l2^3 + 720*t^8*m^13*l1^12*l2^3 - 144*t^6*m^15*l1^12*l2^3 - 112*t^14*m^6*l1^13*l2^3 + 32*t^13*m^7*l1^13*l2^3 + 384*t^12*m^8*l1^13*l2^3 - 32*t^11*m^9*l1^13*l2^3 - 544*t^10*m^10*l1^13*l2^3 - 32*t^9*m^11*l1^13*l2^3 + 384*t^8*m^12*l1^13*l2^3 + 32*t^7*m^13*l1^13*l2^3 - 112*t^6*m^14*l1^13*l2^3 + 88*t^13*m^6*l1^14*l2^3 - 528*t^11*m^8*l1^14*l2^3 + 440*t^10*m^9*l1^14*l2^3 + 440*t^9*m^10*l1^14*l2^3 - 528*t^8*m^11*l1^14*l2^3 + 88*t^6*m^13*l1^14*l2^3 + 8*t^12*m^6*l1^15*l2^3 - 16*t^11*m^7*l1^15*l2^3 - 8*t^10*m^8*l1^15*l2^3 + 32*t^9*m^9*l1^15*l2^3 - 8*t^8*m^10*l1^15*l2^3 - 16*t^7*m^11*l1^15*l2^3 + 8*t^6*m^12*l1^15*l2^3 + 48*t^18*m^10*l1^4*l2^4 - 192*t^16*m^12*l1^4*l2^4 + 288*t^14*m^14*l1^4*l2^4 - 

In [ ]:
Segre_Num_quot(2,2,2)

In [ ]:
all_check(2,2)

checking segre sym: n=2, N=1, r=1
n=2, N=1, r=1 already checked
checking segre sym: n=2, N=1, r=2
n=2, N=1, r=2 already checked
checking segre sym: n=2, N=2, r=2


In [ ]:
all_check(1,3)

checking segre sym: n=1, N=1, r=2
n=1, N=1, r=2 already checked
checking segre sym: n=1, N=1, r=3
n=1, N=1, r=3 already checked
checking segre sym: n=1, N=2, r=3
n=1, N=2, r=3 already checked
all checked for: n=1
checking segre sym: n=2, N=1, r=2
n=2, N=1, r=2 already checked
checking segre sym: n=2, N=1, r=3
n=2, N=1, r=3 already checked
checking segre sym: n=2, N=2, r=3
n=2, N=2, r=3 already checked
all checked for: n=2
checking segre sym: n=3, N=1, r=2
n=3, N=1, r=2 already checked
checking segre sym: n=3, N=1, r=3
n=3, N=1, r=3 already checked
checking segre sym: n=3, N=2, r=3


In [76]:
total_segre_quot(fixed_locus(3,2)[1],2)

Multivariate Polynomial Ring in t, m, l1, l2, l3, l4, l5, w1, w2, w3, w4, w5 over Integer Ring


[[1],
 [1,
  l2 + w1 + 1,
  l2 + w2 + 1,
  -m + l2 + w1 + 1,
  -m + l2 + w2 + 1,
  -t + l2 + w1 + 1,
  -t + l2 + w2 + 1]]

In [57]:
S1=Segre_Num_quot(2,1,2)
S2=Segre_Num_quot(2,2,1)
diff=S1*(-1)^(2)-symmetrize(S2)*(-1)^(2)


B=1
B.factor()
dict(B.factor())

(l1 - t + w1 + 1)*(l1 - t + w2 + 1)*(l1 + w1 + 1)*(l1 + w2 + 1)
(l1 - m + w1 + 1)*(l1 - m + w2 + 1)*(l1 + w1 + 1)*(l1 + w2 + 1)
-1/2*(2*m^2 + 3*m*t + t^2)/((m^3*t - m^2*t^2)*(l1 - m + w1 + 1)*(l1 - m + w2 + 1)*(l1 + w1 + 1)*(l1 + w2 + 1)) + 1/2*(m^2 + 3*m*t + 2*t^2)/((m^2*t^2 - m*t^3)*(l1 - t + w1 + 1)*(l1 - t + w2 + 1)*(l1 + w1 + 1)*(l1 + w2 + 1))
1
(l2 - t + w1 + 1)*(l2 + w1 + 1)
1
(l2 - m + w1 + 1)*(l2 + w1 + 1)
l1 + w1 + 1
(l1 + w1 + 1)*(l2 + w1 + 1)
(l1 - t + w1 + 1)*(l1 + w1 + 1)
(l1 - t + w1 + 1)*(l1 + w1 + 1)
(l1 - m + w1 + 1)*(l1 + w1 + 1)
(l1 - m + w1 + 1)*(l1 + w1 + 1)
-1/2*(2*m^2 + 3*m*t + t^2)/((l1^2*m^3*t - 2*l1*l2*m^3*t + l2^2*m^3*t - l1*m^4*t + l2*m^4*t - l1^2*m^2*t^2 + 2*l1*l2*m^2*t^2 - l2^2*m^2*t^2 + l1*m^3*t^2 - l2*m^3*t^2)*(l1 - m + w1 + 1)*(l1 + w1 + 1)) + 1/2*(m^2 + 3*m*t + 2*t^2)/((l1^2*m^2*t^2 - 2*l1*l2*m^2*t^2 + l2^2*m^2*t^2 - l1^2*m*t^3 + 2*l1*l2*m*t^3 - l2^2*m*t^3 - l1*m^2*t^3 + l2*m^2*t^3 + l1*m*t^4 - l2*m*t^4)*(l1 - t + w1 + 1)*(l1 + w1 + 1)) - (l1^2*m^2 - 

{}

In [38]:
pretty(Segre_Num_quot(1,2,1))


-(m + t)/((l1 - l2)*(l2 + w1 + 1)*m*t)
(m + t)/((l1 - l2)*(l1 + w1 + 1)*m*t)


-(m + t)/((l1 + w1 + 1)*(l2 + w1 + 1)*m*t)

In [19]:
fixed_locus(3,2)[1]

([], [2, 1])

In [14]:
open('segre2d-out.txt', 'w').close()

In [ ]:
all_check(1,4)

checking segre sym: n=1, N=1, r=2
n=1, N=1, r=2 already checked
checking segre sym: n=1, N=1, r=3
n=1, N=1, r=3 already checked
checking segre sym: n=1, N=2, r=3
n=1, N=2, r=3 already checked
all checked for: n=1
checking segre sym: n=2, N=1, r=2
n=2, N=1, r=2 already checked
checking segre sym: n=2, N=1, r=3
n=2, N=1, r=3 already checked
checking segre sym: n=2, N=2, r=3
n=2, N=2, r=3 already checked
all checked for: n=2
checking segre sym: n=3, N=1, r=2
n=3, N=1, r=2 already checked
checking segre sym: n=3, N=1, r=3
n=3, N=1, r=3 already checked
checking segre sym: n=3, N=2, r=3
-1/6*(m + 3*t)*(m + 2*t)*(m + t)/((l1 - l2 + 2*t)*(l1 - l2 + t)*(l1 - l2)*(l2 - t + w1 + 1)*(l2 - t + w2 + 1)*(l2 - t + w3 + 1)*(l2 - 2*t + w1 + 1)*(l2 - 2*t + w2 + 1)*(l2 - 2*t + w3 + 1)*(l2 + w1 + 1)*(l2 + w2 + 1)*(l2 + w3 + 1)*(m - t)*(m - 2*t)*m*t^3)
(2*m + t)*(m + 2*t)*(m + t)/((l1 - l2 + m)*(l1 - l2 + t)*(l1 - l2)*(l2 - m + w1 + 1)*(l2 - m + w2 + 1)*(l2 - m + w3 + 1)*(l2 - t + w1 + 1)*(l2 - t + w2 + 1)*

In [16]:

all_check(4)

checking: n+N+r=4
checking segre sym: n=1, N=1, r=2
n=1, N=1, r=2 already checked
all checked for: n+N+r=4


In [17]:
all_check(5)

checking: n+N+r=5
checking segre sym: n=1, N=1, r=3
n=1, N=1, r=3 already checked
checking segre sym: n=2, N=1, r=2
n=2, N=1, r=2 already checked
all checked for: n+N+r=5


In [18]:
all_check(6)

checking: n+N+r=6
checking segre sym: n=1, N=1, r=4
n=1, N=1, r=4 already checked
checking segre sym: n=1, N=2, r=3
n=1, N=2, r=3 already checked
checking segre sym: n=2, N=1, r=3
n=2, N=1, r=3 already checked
checking segre sym: n=3, N=1, r=2
n=3, N=1, r=2 already checked
all checked for: n+N+r=6


In [19]:
all_check(7)

checking: n+N+r=7
checking segre sym: n=1, N=1, r=5
n=1, N=1, r=5 already checked
checking segre sym: n=1, N=2, r=4
n=1, N=2, r=4 already checked
checking segre sym: n=2, N=1, r=4
n=2, N=1, r=4 already checked
checking segre sym: n=2, N=2, r=3
n=2, N=2, r=3 already checked
checking segre sym: n=3, N=1, r=3
n=3, N=1, r=3 already checked
checking segre sym: n=4, N=1, r=2
n=4, N=1, r=2 already checked
all checked for: n+N+r=7


In [4]:
all_check(8)

checking: n+N+r=8
checking segre sym: n=1, N=2, r=5
n=1, N=2, r=5 already checked
checking segre sym: n=1, N=3, r=4
n=1, N=3, r=4 already checked
checking segre sym: n=2, N=1, r=5
n=2, N=1, r=5 already checked
checking segre sym: n=2, N=2, r=4


KeyboardInterrupt: 

In [3]:
all_check(9)

checking: n+N+r=9
checking segre sym: n=1, N=3, r=5
n=1, N=3, r=5 already checked
checking segre sym: n=2, N=2, r=5


KeyboardInterrupt: 

In [ ]:
all_check(10)

In [ ]:
all_check(11)

In [ ]:
all_check(12)

In [ ]:
all_check(13)

In [ ]:
all_check(14)

In [ ]:
all_check(15)

In [231]:
deg0(set_exp(Verlinde_Num_quot(2,1,2)-symmetrizeK(Verlinde_Num_quot(2,2,1)))).numerator().factor_list()

[(m*t + 2*w1^2 - 4*w1*w2 + 2*w2^2, 1), (m + t, 2), (-1, 1)]

In [188]:
with open("symmetry2d-segre-checked.txt") as f:
        lines = f.read().splitlines()
len(lines)

2

In [298]:
Verlinde_Num_quot_inv(3,4)

0

In [95]:
pretty(Chern_Num_quot(2,2))

1/2*(m + t)^2/(m^2*t^2)

In [70]:
pretty(Chern_Num_quot(3,2))

1/6*(m + t)^3/(m^3*t^3)

In [88]:
pretty(Chern_Num_quot(4,2))

1/24*(m + t)^4/(m^4*t^4)

In [346]:
pretty(Chern_Num_quot(1,4,3))

(m + t)/(m*t)

In [295]:
pretty(Chern_Num_quot(3,1)(l1=0,l2=0,w1=0,w2=0))

1/6*(12*m^4*t^2 + 22*m^3*t^3 + 12*m^2*t^4 - 24*m^3*t^2 - 24*m^2*t^3 - 6*m^3*t - 2*m^2*t^2 - 6*m*t^3 + 6*m^2*t + 6*m*t^2 + m^2 + 2*m*t + t^2)*(m + t)/(m^3*t^3)

In [385]:
pretty(Chern_Num_quot(3,2))

([], [3])
(l2 - t + w1)*(l2 - 2*t + w1)*(l2 + w1)
([], [2, 1])
(l2 - m + w1)*(l2 - t + w1)*(l2 + w1)
([], [1, 1, 1])
(l2 - m + w1)*(l2 - 2*m + w1)*(l2 + w1)
([1], [2])
(l1 + w1)*(l2 - t + w1)*(l2 + w1)
([1], [1, 1])
(l1 + w1)*(l2 - m + w1)*(l2 + w1)
([2], [1])
(l1 - t + w1)*(l1 + w1)*(l2 + w1)
([1, 1], [1])
(l1 - m + w1)*(l1 + w1)*(l2 + w1)
([3], [])
(l1 - t + w1)*(l1 - 2*t + w1)*(l1 + w1)
([2, 1], [])
(l1 - m + w1)*(l1 - t + w1)*(l1 + w1)
([1, 1, 1], [])
(l1 - m + w1)*(l1 - 2*m + w1)*(l1 + w1)


1/6*(m + t)^3/(m^3*t^3)

# RANK 2


In [259]:
pretty(Verlinde_Num_quot(2,1))(x=exp(t),y=exp(m),z=exp(l))

-v1^2*v2^2*z1^4*(e^(3*m) - e^(4*m + 2*t) - e^(3*m + 3*t) - e^(2*m + 4*t) + e^(2*m + 2*t) + e^(2*m + t) + e^(m + 2*t) - e^m + e^(3*t) - e^t)*(e^(m + t) - 1)*e^(-2*m - 2*t)/((e^m + 1)*(e^m - 1)^2*(e^t + 1)*(e^t - 1)^2)

In [23]:
pretty(deg0(pretty(Verlinde_Num_quot_2(2,1))(x=exp(t),y=exp(m),z=exp(l)))-deg0(pretty(Segre_Num_quot_2(2, 1))))

1/12*(m*t - 6*v^2 - 6*w^2)*(m + t)^2/(m^2*t^2)

In [24]:
pretty(deg0(pretty(Verlinde_Num_quot_2(2,2))(x=exp(t),y=exp(m),z=exp(l)))-deg0(pretty(Segre_Num_quot_2(2, 2))))

-1/4*(2*l^2 - 3*m*t + 12*l*v + 14*v^2 + 12*l*w - 4*v*w + 14*w^2)*(m + t)^2/(m^2*t^2)

In [63]:
pretty(deg0(pretty(Verlinde_Num_quot_2(2,3))(x=exp(t),y=exp(m),z=exp(l)))-deg0(pretty(Segre_Num_quot_2(2, 3))))

-3*(l^2 - m*t + 6*l*v + 4*v^2 + 6*l*w - 2*v*w + 4*w^2)*(m + t)^2/(m^2*t^2)

In [33]:
pretty(deg0(pretty(Verlinde_Num_quot_2(3,1))(x=exp(t),y=exp(m),z=exp(l)))+deg0(pretty(Segre_Num_quot_2(3,1))))

-1/48*(72*m^3*t + 83*m^2*t^2 + 72*m*t^3 - 150*m^2*t*v - 150*m*t^2*v + 114*m*t*v^2 + 44*m*v^3 + 44*t*v^3 - 150*m^2*t*w - 150*m*t^2*w + 36*m*t*v*w + 36*m*v^2*w + 36*t*v^2*w + 114*m*t*w^2 + 36*m*v*w^2 + 36*t*v*w^2 + 44*m*w^3 + 44*t*w^3)*(m + t)^2/(m^3*t^3)

In [62]:
pretty(deg0(pretty(Verlinde_Num_quot(3,2))(x=exp(t),y=exp(m),z=exp(l)))-deg0(pretty(Segre_Num_quot(3,2))))

-1/12*(2*l^3*m + 2*l^3*t + 3*l^2*m*t - 42*l*m^2*t + 12*m^3*t - 42*l*m*t^2 - 7*m^2*t^2 + 12*m*t^3 + 24*l^2*m*w + 24*l^2*t*w + 66*l*m*t*w - 84*m^2*t*w - 84*m*t^2*w + 60*l*m*w^2 + 60*l*t*w^2 + 66*m*t*w^2 + 40*m*w^3 + 40*t*w^3)*(m + t)^2/(m^3*t^3)

In [394]:
pretty(Chern_Num_quot(1,3))

(m + t)/(m*t)

In [269]:
pretty(Chern_Num_quot(2,3))

1/2*(m + t)^2/(m^2*t^2)

In [61]:
pretty(Chern_Num_quot_2(3,3))

1/6*(m + t)^3/(m^3*t^3)

In [393]:
pretty(Chern_Num_quot(1,4))

0

In [397]:
pretty(Chern_Num_quot(1,5))

0

In [ ]:
with open("segre2d-out2.txt") as f:
    lines = f.read().splitlines()
A=lines[1]
span = 2
words1 = A.split(" ")
B= [" ".join(words1[i:i+span]) for i in range(1, len(words1), span)]
AA={-1*SR(term)/9216 for term in B}

In [ ]:
with open("segre2d-out1.txt") as f:
    lines = f.read().splitlines()
C=lines[1]
span = 2
words2 = C.split(" ")
D= [" ".join(words2[i:i+span]) for i in range(1, len(words2), span)]
BB={SR(term) for term in D}